In [1]:

    
#scalar * anything = anything
assert compute_type((), ()) == ((), 1)
m = MultiVector(4)
for c in m.vector:
    assert compute_type((), c) == (c, 1)
assert compute_type((1,2), ()) == ((1, 2), 1)
assert compute_type((1,), (1,2)) == ((2,), 1)
assert compute_type((1,), (1,)) == ((), 1)
assert compute_type((2,), (2,)) == ((), 1)
assert compute_type((3,), (1,2)) == ((1,2,3), 1)
assert compute_type((3,), (2,3)) == ((2,), -1)
assert compute_type((1,2), (1,2)) == ((), -1)
assert compute_type((1,2), (2,3)) == ((1,3), 1)
assert compute_type((1,2,3), (2,3)) == ((1,), -1)
assert compute_type((2,3), (2,)) == ((3,), -1)
assert compute_type((2,3), (1,2)) == ((1,3), -1)
assert compute_type((1,4,2), (1,4,2)) == ((), -1) 


assert compute_type((2,3), (1,2,3)) == ((1,), 1)
assert compute_type((2,4), (1,2,3)) == ((1,3,4), 1)

assert compute_type((1,2,3), (1,2)) == ((3,), -1)
assert compute_type((1,2,3), (1,2,3)) == ((), -1)

NameError: name 'compute_type' is not defined

In [60]:
import numpy as np

def compute_geometric(c1, c2):
    """Given a pair of multivector types, return the type of the
    geometric product and its sign. For example:
    () * (2) = (2), 1
    (2) * (2) = (), 1
    (2, 3) * (1, 2) = (3, 1), 1
    (2, 3) * (1, 2, 3) = (1), -1"""
    s = list(c1) + list(c2)
    sign = 1
    
    permuted = True
    while permuted:  
        permuted = False
        t = []        
        for i in range(len(s)):            
            if len(t)>0:
                # e_i * e_i = 1
                if  s[i]==t[-1]:   
                    # remove the duplicate
                    t = t[:-1] 
                # e_j * e_i = -(e_i * e_j)
                elif s[i]<t[-1]:
                    # swap and flip sign
                    t = t[:-1] + [s[i], t[-1]]
                    sign *= -1               
                    permuted = True
                else:
                    # in order, leave alone
                    t.append(s[i])
            else:
                t.append(s[i])        
        s = t
    return tuple(s), sign


def compute_outer(c1, c2):
    """Given a pair of multivector types, return the type of the
    outer product and its sign. For example:
    () ^ (2) = (2), 1
    (2) ^ (2) = 0
    (2) ^ (1) = (1, 2), -1
    (1, 2) ^ (3) = (1,2,3), 1    
    """
    s = list(c1) + list(c2)
    # duplicates become 0
    if len(s)!=len(set(s)):
        return (), 0
    
    sign = 1
    
    permuted = True
    while permuted:  
        permuted = False
        t = []        
        for i in range(len(s)):            
            if len(t)>0:             
                if s[i]<t[-1]:
                    # swap and flip sign
                    t = t[:-1] + [s[i], t[-1]]
                    sign *= -1               
                    permuted = True
                else:
                    # in order, leave alone
                    t.append(s[i])
            else:
                t.append(s[i])        
        s = t
    return tuple(s), sign
        
def blade_name(v):
    if len(v)>0:
        return "e"+"".join([f"{k+1}" for k in v])    
    else:
        return "1"
    

assert blade_name(()) == '1'
assert blade_name((0,)) == 'e1'
assert blade_name((1,)) == 'e2'
assert blade_name((2,)) == 'e3'
assert blade_name((3,)) == 'e4'
assert blade_name((0,1)) == 'e12'
assert blade_name((2,3)) == 'e34'
assert blade_name((3,2)) == 'e43'
assert blade_name((0,3,2)) == 'e143'
assert blade_name((2,0,1)) == 'e312'

#scalar * anything = anything
assert compute_geometric((), ()) == ((), 1)
assert compute_geometric((1,), ()) == ((1,), 1)
assert compute_geometric((1,2,3), ()) == ((1,2,3), 1)
assert compute_geometric((1,2), ()) == ((1,2), 1)

assert compute_geometric((1,2), ()) == ((1, 2), 1)
assert compute_geometric((1,), (1,2)) == ((2,), 1)
assert compute_geometric((1,), (1,)) == ((), 1)
assert compute_geometric((2,), (2,)) == ((), 1)
assert compute_geometric((3,), (1,2)) == ((1,2,3), 1)
assert compute_geometric((3,), (2,3)) == ((2,), -1)
assert compute_geometric((1,2), (1,2)) == ((), -1)
assert compute_geometric((1,2), (2,3)) == ((1,3), 1)
assert compute_geometric((1,2,3), (2,3)) == ((1,), -1)
assert compute_geometric((2,3), (2,)) == ((3,), -1)
assert compute_geometric((2,3), (1,2)) == ((1,3), -1)
assert compute_geometric((1,4,2), (1,4,2)) == ((), -1) 
assert compute_geometric((2,3), (1,2,3)) == ((1,), -1)
assert compute_geometric((2,4), (1,2,3)) == ((1,3,4), 1)
assert compute_geometric((1,2,3), (1,2)) == ((3,), -1)
assert compute_geometric((1,2,3), (1,2,3)) == ((), -1)


In [62]:
import itertools

class MultiVector:
    def __init__(self, n):
        
        components = []
        for i in range(n+1):
            components += list(itertools.combinations(range(n), i))
        self.n = n
        self.vector = {axes:0.0 for axes in components}
        self.multiplication_table = {(blade_i, blade_j):compute_geometric(blade_i, blade_j)
                                     for blade_i in components for blade_j in components}
        self.outer_table = {(blade_i, blade_j):compute_outer(blade_i, blade_j)
                                     for blade_i in components for blade_j in components}
        
    def __repr__(self):
        s = f"MV({self.n}): "
        seq = []
        for k,v in self.vector.items():
            if v!=0:
                seq.append(f"{v:.2f}{blade_name(k)}")
        return s + " + ".join(seq)        
    
    def copy(self):
        other = MultiVector(self.n)
        other.vector = self.vector.copy()
        return other
    
    def __add__(self, other):
        # add
        added = MultiVector(self.n)
        added.vector = {k:self.vector[k]+other.vector[k] for k in self.vector}
        return added
    
    def __sub__(self, other):
        # subtract
        added = MultiVector(self.n)
        added.vector = {k:self.vector[k]-other.vector[k] for k in self.vector}
        return added
    
    def __invert__(self):
        # reversion
        other = self.copy()
        other.vector = {k:-v if k!=() else v for k,v in self.vector.items()}
        return other
    
    def __abs__(self):
        # scalar element of A * ~A
        return (self * (~self)).grade(0)[()]
    
    def grade(self, n):
        return {k:v for k, v in self.vector.items() if len(k)==n}
    
    @classmethod
    def from_vector(self, v):        
        # set the grade 1 elements only
        mv = MultiVector(len(v))
        for i in range(len(v)):
            mv.vector[(i,)] = v[i]
        print(mv.vector)
        return mv
        
    def reflect_about(self, other):
        return other * self * other    
    
    def rotate(self, other):
        return self * (other * ~self)

    # outer product
    def __xor__(self, other):
        # outer product
        product = MultiVector(self.n)
        for blade_i, v_i in self.vector.items():
            for blade_j, v_j in other.vector.items():                
                blade, sign = self.outer_table[(blade_i, blade_j)]                     
                product.vector[blade] += v_i * v_j * sign
        return product

    # geometric product
    def __mul__(self, other):
        product = MultiVector(self.n)
        # scalar multiplication (promote first)
        if type(other)==float or type(other)==int:
            mv_other = MultiVector(self.n)
            mv_other.vector[()] = other
            other = mv_other                                
            
        if type(other)==MultiVector:
            # geometric product
            for blade_i, v_i in self.vector.items():
                for blade_j, v_j in other.vector.items():
                    blade, sign = self.multiplication_table[(blade_i, blade_j)]                    
                    product.vector[blade] += v_i * v_j * sign
        return product
                
    
    def __eq__(self, other):
        return self.vector == other.vector
        
    def print_table(self):
        # what a  mess
        print("\t", end='')
        for j, blade_j in enumerate(self.vector):
            print(blade_name(blade_j), end='\t')
        print()
        for i, blade_i in enumerate(self.vector):    
            
            for j, blade_j in enumerate(self.vector):
                if j==0:
                    print(blade_name(blade_i), end="\t")        
                    
                entry = self.multiplication_table[(blade_i, blade_j)]
                name = blade_name(entry[0])
                if entry[1]<0:
                    name = '-'+name
                print(name, end='\t')
            print()
            
                
        
        
m2 = MultiVector(4)
m2.print_table()
print(m2)
def normed(x):
    return x / np.linalg.norm(x)

v1 = MultiVector.from_vector([1,3,-2])
v2 = MultiVector.from_vector([5,2,8])


b1 = MultiVector.from_vector([0,1,0])
b2 = MultiVector.from_vector([1,0,0])
bivector = b1 ^ b2
print(bivector)
v1 = MultiVector.from_vector([1,3,-2])
r_v = MultiVector.get_rotor(0.1, bivector)
r_v.rotate(v1)

	1	e1	e2	e3	e4	e12	e13	e14	e23	e24	e34	e123	e124	e134	e234	e1234	
1	1	e1	e2	e3	e4	e12	e13	e14	e23	e24	e34	e123	e124	e134	e234	e1234	
e1	e1	1	e12	e13	e14	e2	e3	e4	e123	e124	e134	e23	e24	e34	e1234	e234	
e2	e2	-e12	1	e23	e24	-e1	-e123	-e124	e3	e4	e234	-e13	-e14	-e1234	e34	-e134	
e3	e3	-e13	-e23	1	e34	e123	-e1	-e134	-e2	-e234	e4	e12	e1234	-e14	-e24	e124	
e4	e4	-e14	-e24	-e34	1	e124	e134	-e1	e234	-e2	-e3	-e1234	e12	e13	e23	-e123	
e12	e12	-e2	e1	e123	e124	-1	-e23	-e24	e13	e14	e1234	-e3	-e4	-e234	e134	-e34	
e13	e13	-e3	-e123	e1	e134	e23	-1	-e34	-e12	-e1234	e14	e2	e234	-e4	-e124	e24	
e14	e14	-e4	-e124	-e134	e1	e24	e34	-1	e1234	-e12	-e13	-e234	e2	e3	e123	-e23	
e23	e23	e123	-e3	e2	e234	-e13	e12	e1234	-1	-e34	e24	-e1	-e134	e124	-e4	-e14	
e24	e24	e124	-e4	-e234	e2	-e14	-e1234	e12	e34	-1	-e23	e134	-e1	-e123	e3	e13	
e34	e34	e134	e234	-e4	e3	e1234	-e14	e13	-e24	e23	-1	-e124	e123	-e1	-e2	-e12	
e123	e123	e23	-e13	e12	e1234	-e3	e2	e234	-e1	-e134	e124	-1	-e34	e24	-e14	-e4	
e124	e124	e24	-e14	-e1234	e12	-

AttributeError: type object 'MultiVector' has no attribute 'get_rotor'

In [19]:
import itertools
list(itertools.combinations([1,2,3],0))

[()]